In [38]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
import requests
import csv

import json
from pandas.io.json import json_normalize

from geopy.geocoders import Nominatim 
from geopy.exc import GeocoderTimedOut

print('Libraries imported.')

Libraries imported.


###### Extract the HTML code of the source website

In [39]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_Sydney_suburbs').text
soup = BeautifulSoup(source, 'lxml')

print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of Sydney suburbs - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"e925255b-0650-463a-a50a-d0c8eccc3196","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_Sydney_suburbs","wgTitle":"List of Sydney suburbs","wgCurRevisionId":955135801,"wgRevisionId":955135801,"wgArticleId":241497,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Use dmy dates from October 2012","Commons category link is on Wikidata","Suburbs of Sydney","Lists of suburbs in Australia","

###### After detailed observation of the HTML code, you can see that all the suburb names are in paragraph tags. So we can extract all the paragraph tags and then filter out the extra, unnecessary data.

In [40]:
paras=[]
for i in soup.select('p'):
    paras.append(i.getText()) 
    print(i.getText())
paras



This is a complete listing of the suburbs and localities in the greater Sydney area in alphabetical order.
Sydney has about 38 local government areas, each consisting of several suburbs (suburbs in Australia are purely geographical, not political, divisions).  See table below, Category: Suburbs of Sydney and Category: Local government areas in Sydney.

Suburbs are listed here if they are inside the Sydney metro area, and are listed in the Geographical Names Register[1]  as being suburbs. For this list, the boundaries of the Sydney metro area are defined as the Hawkesbury/Nepean River in the north/north west, and then the outer boundaries of the City of Penrith, Camden Council, the City of Campbelltown and Sutherland Shire.

Some but not all Sydney localities are also listed, and localities are shown in italics to differentiate them from suburbs. Further localities may be added if they are on the Geographical Names Register, are inside the Sydney metro area, and are also listed in the

['\n',
 'This is a complete listing of the suburbs and localities in the greater Sydney area in alphabetical order.\nSydney has about 38 local government areas, each consisting of several suburbs (suburbs in Australia are purely geographical, not political, divisions).  See table below, Category: Suburbs of Sydney and Category: Local government areas in Sydney.\n',
 'Suburbs are listed here if they are inside the Sydney metro area, and are listed in the Geographical Names Register[1]  as being suburbs. For this list, the boundaries of the Sydney metro area are defined as the Hawkesbury/Nepean River in the north/north west, and then the outer boundaries of the City of Penrith, Camden Council, the City of Campbelltown and Sutherland Shire.\n',
 'Some but not all Sydney localities are also listed, and localities are shown in italics to differentiate them from suburbs. Further localities may be added if they are on the Geographical Names Register, are inside the Sydney metro area, and are 

###### Find the index from where the necessary data starts and ends and remove all the unnecessary data.

In [41]:
paras[0:6]

['\n',
 'This is a complete listing of the suburbs and localities in the greater Sydney area in alphabetical order.\nSydney has about 38 local government areas, each consisting of several suburbs (suburbs in Australia are purely geographical, not political, divisions).  See table below, Category: Suburbs of Sydney and Category: Local government areas in Sydney.\n',
 'Suburbs are listed here if they are inside the Sydney metro area, and are listed in the Geographical Names Register[1]  as being suburbs. For this list, the boundaries of the Sydney metro area are defined as the Hawkesbury/Nepean River in the north/north west, and then the outer boundaries of the City of Penrith, Camden Council, the City of Campbelltown and Sutherland Shire.\n',
 'Some but not all Sydney localities are also listed, and localities are shown in italics to differentiate them from suburbs. Further localities may be added if they are on the Geographical Names Register, are inside the Sydney metro area, and are 

In [42]:
del paras[0:5]

In [43]:
paras[0]

'Abbotsbury - \nAbbotsford - \nAcacia Gardens - \nAgnes Banks - \nAirds - \nAlexandria - \nAlfords Point -\nAllambie Heights - \nAllawah - \nAmbarvale - \nAnnandale - \nAnnangrove - \nArcadia - \nArncliffe - \nArndell Park - \nArtarmon - \nAshbury - \nAshcroft - \nAshfield - \nAsquith - \nAuburn -\nAustral - \nAvalon Beach\n'

###### From this cell onwards, the extra parts of the data obtained such as the hyphens and the '/n' parts of the HTML code are removed to get pure data.

In [44]:
i=0
suburbs=[]
for i in range(len(paras)):
    word = paras[i]
    word.split(' - \n')
    suburbs.append(word.split(' - \n'))
suburbs

[['Abbotsbury',
  'Abbotsford',
  'Acacia Gardens',
  'Agnes Banks',
  'Airds',
  'Alexandria',
  'Alfords Point -\nAllambie Heights',
  'Allawah',
  'Ambarvale',
  'Annandale',
  'Annangrove',
  'Arcadia',
  'Arncliffe',
  'Arndell Park',
  'Artarmon',
  'Ashbury',
  'Ashcroft',
  'Ashfield',
  'Asquith',
  'Auburn -\nAustral',
  'Avalon Beach\n'],
 ['Badgerys Creek',
  'Balgowlah',
  'Balgowlah Heights',
  'Balmain',
  'Balmain East',
  'Bangor',
  'Banksia',
  'Banksmeadow',
  'Bankstown',
  'Bankstown Aerodrome -\nBarangaroo',
  'Barden Ridge',
  'Bardia',
  'Bardwell Park',
  'Bardwell Valley',
  'Bass Hill',
  'Baulkham Hills',
  'Bayview',
  'Beacon Hill',
  'Beaconsfield',
  'Beaumont Hills -\nBeecroft',
  'Belfield',
  'Bella Vista',
  'Bellevue Hill',
  'Belmore',
  'Belrose',
  'Berala',
  'Berkshire Park',
  'Berowra',
  'Berowra Creek -\nBerowra Heights',
  'Berowra Waters',
  'Berrilee',
  'Beverley Park',
  'Beverly Hills',
  'Bexley',
  'Bexley North',
  'Bickley Vale -

In [45]:
i=0
suburbs_1=[]
for i in range(len(suburbs)):
    j=0
    for j in range(len(suburbs[i])):
        word = suburbs[i][j]
        word.split(' -\n')
        suburbs_1.append(word.split(' -\n'))
        print(word.split(' -\n'))
suburbs_1

['Abbotsbury']
['Abbotsford']
['Acacia Gardens']
['Agnes Banks']
['Airds']
['Alexandria']
['Alfords Point', 'Allambie Heights']
['Allawah']
['Ambarvale']
['Annandale']
['Annangrove']
['Arcadia']
['Arncliffe']
['Arndell Park']
['Artarmon']
['Ashbury']
['Ashcroft']
['Ashfield']
['Asquith']
['Auburn', 'Austral']
['Avalon Beach\n']
['Badgerys Creek']
['Balgowlah']
['Balgowlah Heights']
['Balmain']
['Balmain East']
['Bangor']
['Banksia']
['Banksmeadow']
['Bankstown']
['Bankstown Aerodrome', 'Barangaroo']
['Barden Ridge']
['Bardia']
['Bardwell Park']
['Bardwell Valley']
['Bass Hill']
['Baulkham Hills']
['Bayview']
['Beacon Hill']
['Beaconsfield']
['Beaumont Hills', 'Beecroft']
['Belfield']
['Bella Vista']
['Bellevue Hill']
['Belmore']
['Belrose']
['Berala']
['Berkshire Park']
['Berowra']
['Berowra Creek', 'Berowra Heights']
['Berowra Waters']
['Berrilee']
['Beverley Park']
['Beverly Hills']
['Bexley']
['Bexley North']
['Bickley Vale', 'Bidwill']
['Bilgola Beach']
['Bilgola Plateau', 'Birchgr

[['Abbotsbury'],
 ['Abbotsford'],
 ['Acacia Gardens'],
 ['Agnes Banks'],
 ['Airds'],
 ['Alexandria'],
 ['Alfords Point', 'Allambie Heights'],
 ['Allawah'],
 ['Ambarvale'],
 ['Annandale'],
 ['Annangrove'],
 ['Arcadia'],
 ['Arncliffe'],
 ['Arndell Park'],
 ['Artarmon'],
 ['Ashbury'],
 ['Ashcroft'],
 ['Ashfield'],
 ['Asquith'],
 ['Auburn', 'Austral'],
 ['Avalon Beach\n'],
 ['Badgerys Creek'],
 ['Balgowlah'],
 ['Balgowlah Heights'],
 ['Balmain'],
 ['Balmain East'],
 ['Bangor'],
 ['Banksia'],
 ['Banksmeadow'],
 ['Bankstown'],
 ['Bankstown Aerodrome', 'Barangaroo'],
 ['Barden Ridge'],
 ['Bardia'],
 ['Bardwell Park'],
 ['Bardwell Valley'],
 ['Bass Hill'],
 ['Baulkham Hills'],
 ['Bayview'],
 ['Beacon Hill'],
 ['Beaconsfield'],
 ['Beaumont Hills', 'Beecroft'],
 ['Belfield'],
 ['Bella Vista'],
 ['Bellevue Hill'],
 ['Belmore'],
 ['Belrose'],
 ['Berala'],
 ['Berkshire Park'],
 ['Berowra'],
 ['Berowra Creek', 'Berowra Heights'],
 ['Berowra Waters'],
 ['Berrilee'],
 ['Beverley Park'],
 ['Beverly Hil

In [46]:
i=0
suburbs=[]
for i in range(len(suburbs_1)):
    j=0
    for j in range(len(suburbs_1[i])):
        word = suburbs_1[i][j]
        word.split(', ')
        suburbs.append(word.split(', '))
suburbs

[['Abbotsbury'],
 ['Abbotsford'],
 ['Acacia Gardens'],
 ['Agnes Banks'],
 ['Airds'],
 ['Alexandria'],
 ['Alfords Point'],
 ['Allambie Heights'],
 ['Allawah'],
 ['Ambarvale'],
 ['Annandale'],
 ['Annangrove'],
 ['Arcadia'],
 ['Arncliffe'],
 ['Arndell Park'],
 ['Artarmon'],
 ['Ashbury'],
 ['Ashcroft'],
 ['Ashfield'],
 ['Asquith'],
 ['Auburn'],
 ['Austral'],
 ['Avalon Beach\n'],
 ['Badgerys Creek'],
 ['Balgowlah'],
 ['Balgowlah Heights'],
 ['Balmain'],
 ['Balmain East'],
 ['Bangor'],
 ['Banksia'],
 ['Banksmeadow'],
 ['Bankstown'],
 ['Bankstown Aerodrome'],
 ['Barangaroo'],
 ['Barden Ridge'],
 ['Bardia'],
 ['Bardwell Park'],
 ['Bardwell Valley'],
 ['Bass Hill'],
 ['Baulkham Hills'],
 ['Bayview'],
 ['Beacon Hill'],
 ['Beaconsfield'],
 ['Beaumont Hills'],
 ['Beecroft'],
 ['Belfield'],
 ['Bella Vista'],
 ['Bellevue Hill'],
 ['Belmore'],
 ['Belrose'],
 ['Berala'],
 ['Berkshire Park'],
 ['Berowra'],
 ['Berowra Creek'],
 ['Berowra Heights'],
 ['Berowra Waters'],
 ['Berrilee'],
 ['Beverley Park'],

In [47]:
i=0
suburbs_1=[]
for i in range(len(suburbs)):
    j=0
    for j in range(len(suburbs[i])):
        word = suburbs[i][j]
        result = word.split('\n')
        suburbs_1.append(result[0])
suburbs_1     

['Abbotsbury',
 'Abbotsford',
 'Acacia Gardens',
 'Agnes Banks',
 'Airds',
 'Alexandria',
 'Alfords Point',
 'Allambie Heights',
 'Allawah',
 'Ambarvale',
 'Annandale',
 'Annangrove',
 'Arcadia',
 'Arncliffe',
 'Arndell Park',
 'Artarmon',
 'Ashbury',
 'Ashcroft',
 'Ashfield',
 'Asquith',
 'Auburn',
 'Austral',
 'Avalon Beach',
 'Badgerys Creek',
 'Balgowlah',
 'Balgowlah Heights',
 'Balmain',
 'Balmain East',
 'Bangor',
 'Banksia',
 'Banksmeadow',
 'Bankstown',
 'Bankstown Aerodrome',
 'Barangaroo',
 'Barden Ridge',
 'Bardia',
 'Bardwell Park',
 'Bardwell Valley',
 'Bass Hill',
 'Baulkham Hills',
 'Bayview',
 'Beacon Hill',
 'Beaconsfield',
 'Beaumont Hills',
 'Beecroft',
 'Belfield',
 'Bella Vista',
 'Bellevue Hill',
 'Belmore',
 'Belrose',
 'Berala',
 'Berkshire Park',
 'Berowra',
 'Berowra Creek',
 'Berowra Heights',
 'Berowra Waters',
 'Berrilee',
 'Beverley Park',
 'Beverly Hills',
 'Bexley',
 'Bexley North',
 'Bickley Vale',
 'Bidwill',
 'Bilgola Beach',
 'Bilgola Plateau',
 'Bi

In [48]:
i=0
suburbs=[]
for i in range(len(suburbs_1)):
    word = suburbs_1[i]
    result = word.split(' -  ' )
    suburbs.append(result)       
suburbs

[['Abbotsbury'],
 ['Abbotsford'],
 ['Acacia Gardens'],
 ['Agnes Banks'],
 ['Airds'],
 ['Alexandria'],
 ['Alfords Point'],
 ['Allambie Heights'],
 ['Allawah'],
 ['Ambarvale'],
 ['Annandale'],
 ['Annangrove'],
 ['Arcadia'],
 ['Arncliffe'],
 ['Arndell Park'],
 ['Artarmon'],
 ['Ashbury'],
 ['Ashcroft'],
 ['Ashfield'],
 ['Asquith'],
 ['Auburn'],
 ['Austral'],
 ['Avalon Beach'],
 ['Badgerys Creek'],
 ['Balgowlah'],
 ['Balgowlah Heights'],
 ['Balmain'],
 ['Balmain East'],
 ['Bangor'],
 ['Banksia'],
 ['Banksmeadow'],
 ['Bankstown'],
 ['Bankstown Aerodrome'],
 ['Barangaroo'],
 ['Barden Ridge'],
 ['Bardia'],
 ['Bardwell Park'],
 ['Bardwell Valley'],
 ['Bass Hill'],
 ['Baulkham Hills'],
 ['Bayview'],
 ['Beacon Hill'],
 ['Beaconsfield'],
 ['Beaumont Hills'],
 ['Beecroft'],
 ['Belfield'],
 ['Bella Vista'],
 ['Bellevue Hill'],
 ['Belmore'],
 ['Belrose'],
 ['Berala'],
 ['Berkshire Park'],
 ['Berowra'],
 ['Berowra Creek'],
 ['Berowra Heights'],
 ['Berowra Waters'],
 ['Berrilee'],
 ['Beverley Park'],
 

In [49]:
i=0
for i in range(len(suburbs)):
    if(len(suburbs[i])==2):
        suburbs[i]=suburbs[i][0]
suburbs

[['Abbotsbury'],
 ['Abbotsford'],
 ['Acacia Gardens'],
 ['Agnes Banks'],
 ['Airds'],
 ['Alexandria'],
 ['Alfords Point'],
 ['Allambie Heights'],
 ['Allawah'],
 ['Ambarvale'],
 ['Annandale'],
 ['Annangrove'],
 ['Arcadia'],
 ['Arncliffe'],
 ['Arndell Park'],
 ['Artarmon'],
 ['Ashbury'],
 ['Ashcroft'],
 ['Ashfield'],
 ['Asquith'],
 ['Auburn'],
 ['Austral'],
 ['Avalon Beach'],
 ['Badgerys Creek'],
 ['Balgowlah'],
 ['Balgowlah Heights'],
 ['Balmain'],
 ['Balmain East'],
 ['Bangor'],
 ['Banksia'],
 ['Banksmeadow'],
 ['Bankstown'],
 ['Bankstown Aerodrome'],
 ['Barangaroo'],
 ['Barden Ridge'],
 ['Bardia'],
 ['Bardwell Park'],
 ['Bardwell Valley'],
 ['Bass Hill'],
 ['Baulkham Hills'],
 ['Bayview'],
 ['Beacon Hill'],
 ['Beaconsfield'],
 ['Beaumont Hills'],
 ['Beecroft'],
 ['Belfield'],
 ['Bella Vista'],
 ['Bellevue Hill'],
 ['Belmore'],
 ['Belrose'],
 ['Berala'],
 ['Berkshire Park'],
 ['Berowra'],
 ['Berowra Creek'],
 ['Berowra Heights'],
 ['Berowra Waters'],
 ['Berrilee'],
 ['Beverley Park'],
 

In [50]:
i=0
for i in range(len(suburbs)):
    j=0
    for j in range(len(suburbs[i])):
        word = suburbs[i][j]
        a=1
        for a in range(len(word)-1):
            if((word[a] =='-')&(word[a-1]==' ')&(word[a+1]==' ')):
                result = word.split('-')
                suburbs[i][j] = result[0]
                suburbs.append(result[1])
suburbs

[['Abbotsbury'],
 ['Abbotsford'],
 ['Acacia Gardens'],
 ['Agnes Banks'],
 ['Airds'],
 ['Alexandria'],
 ['Alfords Point'],
 ['Allambie Heights'],
 ['Allawah'],
 ['Ambarvale'],
 ['Annandale'],
 ['Annangrove'],
 ['Arcadia'],
 ['Arncliffe'],
 ['Arndell Park'],
 ['Artarmon'],
 ['Ashbury'],
 ['Ashcroft'],
 ['Ashfield'],
 ['Asquith'],
 ['Auburn'],
 ['Austral'],
 ['Avalon Beach'],
 ['Badgerys Creek'],
 ['Balgowlah'],
 ['Balgowlah Heights'],
 ['Balmain'],
 ['Balmain East'],
 ['Bangor'],
 ['Banksia'],
 ['Banksmeadow'],
 ['Bankstown'],
 ['Bankstown Aerodrome'],
 ['Barangaroo'],
 ['Barden Ridge'],
 ['Bardia'],
 ['Bardwell Park'],
 ['Bardwell Valley'],
 ['Bass Hill'],
 ['Baulkham Hills'],
 ['Bayview'],
 ['Beacon Hill'],
 ['Beaconsfield'],
 ['Beaumont Hills'],
 ['Beecroft'],
 ['Belfield'],
 ['Bella Vista'],
 ['Bellevue Hill'],
 ['Belmore'],
 ['Belrose'],
 ['Berala'],
 ['Berkshire Park'],
 ['Berowra'],
 ['Berowra Creek'],
 ['Berowra Heights'],
 ['Berowra Waters'],
 ['Berrilee'],
 ['Beverley Park'],
 

In [51]:
suburbs_new=[]
i=0
for i in range(len(suburbs)):
    if type(suburbs[i]) is list:
        suburbs_new.append(suburbs[i][0])
    else:
        suburbs_new.append(suburbs[i])
suburbs_new 

['Abbotsbury',
 'Abbotsford',
 'Acacia Gardens',
 'Agnes Banks',
 'Airds',
 'Alexandria',
 'Alfords Point',
 'Allambie Heights',
 'Allawah',
 'Ambarvale',
 'Annandale',
 'Annangrove',
 'Arcadia',
 'Arncliffe',
 'Arndell Park',
 'Artarmon',
 'Ashbury',
 'Ashcroft',
 'Ashfield',
 'Asquith',
 'Auburn',
 'Austral',
 'Avalon Beach',
 'Badgerys Creek',
 'Balgowlah',
 'Balgowlah Heights',
 'Balmain',
 'Balmain East',
 'Bangor',
 'Banksia',
 'Banksmeadow',
 'Bankstown',
 'Bankstown Aerodrome',
 'Barangaroo',
 'Barden Ridge',
 'Bardia',
 'Bardwell Park',
 'Bardwell Valley',
 'Bass Hill',
 'Baulkham Hills',
 'Bayview',
 'Beacon Hill',
 'Beaconsfield',
 'Beaumont Hills',
 'Beecroft',
 'Belfield',
 'Bella Vista',
 'Bellevue Hill',
 'Belmore',
 'Belrose',
 'Berala',
 'Berkshire Park',
 'Berowra',
 'Berowra Creek',
 'Berowra Heights',
 'Berowra Waters',
 'Berrilee',
 'Beverley Park',
 'Beverly Hills',
 'Bexley',
 'Bexley North',
 'Bickley Vale',
 'Bidwill',
 'Bilgola Beach',
 'Bilgola Plateau',
 'Bi

In [52]:
for i in range(len(suburbs_new)):
    s = suburbs_new[i] 
    if (s[0] == ' '):
        suburbs_new[i] = s[1:]
suburbs_new

['Abbotsbury',
 'Abbotsford',
 'Acacia Gardens',
 'Agnes Banks',
 'Airds',
 'Alexandria',
 'Alfords Point',
 'Allambie Heights',
 'Allawah',
 'Ambarvale',
 'Annandale',
 'Annangrove',
 'Arcadia',
 'Arncliffe',
 'Arndell Park',
 'Artarmon',
 'Ashbury',
 'Ashcroft',
 'Ashfield',
 'Asquith',
 'Auburn',
 'Austral',
 'Avalon Beach',
 'Badgerys Creek',
 'Balgowlah',
 'Balgowlah Heights',
 'Balmain',
 'Balmain East',
 'Bangor',
 'Banksia',
 'Banksmeadow',
 'Bankstown',
 'Bankstown Aerodrome',
 'Barangaroo',
 'Barden Ridge',
 'Bardia',
 'Bardwell Park',
 'Bardwell Valley',
 'Bass Hill',
 'Baulkham Hills',
 'Bayview',
 'Beacon Hill',
 'Beaconsfield',
 'Beaumont Hills',
 'Beecroft',
 'Belfield',
 'Bella Vista',
 'Bellevue Hill',
 'Belmore',
 'Belrose',
 'Berala',
 'Berkshire Park',
 'Berowra',
 'Berowra Creek',
 'Berowra Heights',
 'Berowra Waters',
 'Berrilee',
 'Beverley Park',
 'Beverly Hills',
 'Bexley',
 'Bexley North',
 'Bickley Vale',
 'Bidwill',
 'Bilgola Beach',
 'Bilgola Plateau',
 'Bi

###### Now that we have a list of the suburbs of Sydney without any garbage and perfectly cleaned, we can get the geographical locations of all the suburbs.

In [53]:
geolocator = Nominatim(user_agent="syd_explorer")
lats=[]
longs=[]
not_done=[]
for i in range(len(suburbs_new)):
    address = suburbs_new[i]+', Sydney, Australia'
    try:
        location = geolocator.geocode(address)
        if location:
            lats.append(location.latitude)
            longs.append(location.longitude)
        if location is None:
            lats.append(np.nan)
            longs.append(np.nan)
    except GeocoderTimedOut as e:
        not_done.append(i)
        lats.append(np.nan)
        longs.append(np.nan)

###### Convert all the names and locations into a dataframe.

In [54]:
sydney_suburbs = pd.DataFrame(columns=["Suburb","Latitude",'Longitude'])
sydney_suburbs['Suburb'] = suburbs_new
sydney_suburbs ['Latitude'] = lats
sydney_suburbs ['Longitude'] = longs
sydney_suburbs.sort_values(by = ["Suburb"], inplace = True)
sydney_suburbs.dropna(inplace = True)
sydney_suburbs.reset_index(inplace = True)
sydney_suburbs.drop(['index'],inplace = True,axis = 1)
sydney_suburbs

,Suburb,Latitude,Longitude
0,Abbotsbury,-33.869285,150.866703
1,Abbotsford,-33.850553,151.129759
2,Acacia Gardens,-33.732459,150.912532
3,Agnes Banks,-33.614508,150.711448
4,Airds,-34.090000,150.826111
...,...,...,...
673,Yarramundi,-33.631817,150.667996
674,Yarrawarrah,-34.056681,151.030888
675,Yennora,-33.864629,150.971150
676,Yowie Bay,-34.050278,151.103333


In [55]:
sydney_suburbs

,Suburb,Latitude,Longitude
0,Abbotsbury,-33.869285,150.866703
1,Abbotsford,-33.850553,151.129759
2,Acacia Gardens,-33.732459,150.912532
3,Agnes Banks,-33.614508,150.711448
4,Airds,-34.090000,150.826111
...,...,...,...
673,Yarramundi,-33.631817,150.667996
674,Yarrawarrah,-34.056681,151.030888
675,Yennora,-33.864629,150.971150
676,Yowie Bay,-34.050278,151.103333


###### Convert the dataframe to a csv file.

In [56]:
sydney_suburbs.to_csv(r'sydney_suburbs.csv', index = False)